# yt-tutorial

scripts: https://github.com/kuochuanpan/yt-tutorial
data: https://goo.gl/dSXTs7

Kuo-Chuan Pan 2018.06.01

In [ ]:
import yt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

change plot style (optional)

In [ ]:
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['legend.handlelength']=4
mpl.rcParams['legend.fontsize']=14
mpl.rcParams['legend.frameon']=False
mpl.rcParams['axes.labelsize']=20
mpl.rcParams['xtick.minor.visible']=True
mpl.rcParams['ytick.minor.visible']=True
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['xtick.labelsize']   = 14
mpl.rcParams['ytick.labelsize']   = 14

Preparing data set

In [ ]:
fn_1d = "./data/ccsn1d_hdf5_chk_0500"
fn_2d = "./data/ccsn2d_hdf5_plt_cnt_0200"
fn_3d = "./data/ccsn3d_hdf5_plt_cnt_0656"

Load data ...

In [ ]:
ds_1d = yt.load(fn_1d)
ds_2d = yt.load(fn_2d)
ds_3d = yt.load(fn_3d)

# yt quick start

## Data Inspection

In [ ]:
ds_3d.current_time.in_cgs()

In [ ]:
ds_3d.current_time.in_units("ms")

In [ ]:
ds_3d.current_time.in_units("day")

In [ ]:
ds_3d.domain_width.in_units("km")

In [ ]:
ds_3d.print_stats()

In [ ]:
ds_3d.field_list

In [ ]:
ds_3d.derived_field_list

In [ ]:
print(ds_3d.field_info["gas", "pressure_gradient_x"].get_source())

In [ ]:
sphere = ds_3d.sphere("max", (500, 'km'))

In [ ]:
sphere

In [ ]:
list(sphere.quantities.keys())

In [ ]:
tot_mass = sphere.quantities.total_mass()

In [ ]:
print(tot_mass.in_units("msun"))

In [ ]:
sphere.quantities.center_of_mass().in_units("km")

## Simple Visualizations

In [ ]:
path="/Users/pan/codes/BANG/object"
fn = path+"/sedov_hdf5_chk_0003"
ds = yt.load(fn)

In [ ]:
slice = yt.SlicePlot(ds,'z',"dens")

In [ ]:
slice.annotate_grids()
slice.show()

In [ ]:
slice = yt.SlicePlot(ds_3d,'z',"entr",width=(250,'km'))

In [ ]:
slice.show()

In [ ]:
slice.set_log('entr',False)
slice.set_cmap('entr',cmap="Spectral_r")
slice.set_zlim('entr',0,35)
slice.show()

In [ ]:
slice = yt.SlicePlot(ds_3d,'z',"radial_velocity",width=(250,'km'))
slice.set_log('radial_velocity',False)
slice.show()

In [ ]:
v, c = ds_3d.find_max("density")
slice.set_center((c[0], c[1]))

In [ ]:
slice.pan_rel((-0.2, 0.2))

In [ ]:
slice.pan_rel((0.2, -0.2))

In [ ]:
slice.annotate_grids()

In [ ]:
slice.zoom(2)

In [ ]:
slice.annotate_velocity()

In [ ]:
slice.annotate_contour("dens",ncont=1, take_log=True, clim=[1.e11,1e11], plot_args={'colors':'w'})

## Projection Plot

In [ ]:
yt.ProjectionPlot(ds_3d, "z", "entr",width=(250,'km')).show()

In [ ]:
proj = yt.ProjectionPlot(ds_3d, "z", "entr",weight_field="density", width=(250,'km'))
proj.set_log('entr',False)
proj.show()

## Off-axis slicing 

In [ ]:
sp = ds_3d.sphere("center", (500, "km"))
L = sp.quantities.angular_momentum_vector()
print("Angular momentum vector: {0}".format(L))
p = yt.OffAxisSlicePlot(ds_3d, L, "entr", sp.center, (400, "km"))

In [ ]:
p.set_width((250,'km'))
p.set_log('entr',False)
p.set_cmap('entr',cmap="Spectral_r")
p.show()

## Phase plot

In [ ]:
sphere = ds_3d.sphere("c", (500.0, "km"))
phase_plot = yt.PhasePlot(sphere, "entr", "ye  ", "cell_mass",
                    weight_field=None)

# Set the units of mass to be in solar masses (not the default in cgs)
phase_plot.set_unit('cell_mass', 'Msun')
phase_plot.set_log('entr',False)
phase_plot.set_log('ye  ',False)
#phase_plot.save()

## Line plot

In [ ]:
# Create a line plot of the variables 'entr' and 'ye . ' with 1000 sampling points evenly spaced
# between the coordinates (0, 0, 0) and (0, 2e7, 0)
line = yt.LinePlot(ds_3d, ['entr', 'ye  '], (0, 0, 0), (2e7, 0, 0), 1000)
line.annotate_legend(['entr','ye  '])
line.show()

if you don't like the default format in LinePlot, ....

In [ ]:
rmax = 1.2e7

In [ ]:
ray = ds_3d.ray([0,0,0],[rmax,0,0])

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(ray['t']*rmax/1.e5,ray['entr'],'-')
plt.ylabel("Entropy [kb/by]")
plt.xlabel("Radius [ckm]")
plt.xlim([0,rmax/1e5])

Can be applied to 1D/2D data as well

In [ ]:
ray1 = ds_1d.ray([0,0,0],[2e7,0,0])
ray2 = ds_2d.ray([0,0,0],[2e7,0,0])
plt.figure(figsize=(8,6))
plt.plot(ray1['t']*2e7/1.e5,ray1['entr'],'-',label="1D data")
plt.plot(ray2['t']*2e7/1.e5,ray2['entr'],'-',label="2D data")
plt.ylabel("Entropy [kb/by]")
plt.xlabel("Radius [ckm]")
plt.legend(loc='best')
plt.xlim([0,2e7/1e5])

## Values at a point (0D)

In [ ]:
values = ds_3d.h.find_field_values_at_point(["dens","entr","ye  "],[2e7,0,0])

In [ ]:
print(values)

# Sedov problem

In [ ]:
path="/Users/pan/codes/BANG/object"
fn = path+"/sedov_hdf5_chk_0003"

In [ ]:
ds = yt.load(fn)

In [ ]:
ray = ds.ray([0.5,0,0],[0.5,1,0])

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(ray['t'],ray['dens'],'-',label="Sedov")
plt.ylabel(r"Density [g/cm$^3$]")
plt.xlabel("Radius [ckm]")
plt.legend(loc='best')
plt.xlim([0,1])

## Profile plot

In [ ]:
sphere = ds_3d.sphere([0,0,0],(2e7,'cm'))

In [ ]:
profile = yt.ProfilePlot(sphere, "radius", ["entr"],n_bins=120)
profile.set_log('radius',False)
profile.set_log('entr',False)
profile.show()

In [ ]:
profile = yt.create_profile(sphere, "radius",["entr","ye  "],n_bins=200,extrema={'radius':(1e5,2e7)},logs={'radius':False},weight_field='cell_mass',accumulation=False)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(profile.x/1e5,profile["entr"].v)
plt.ylabel("Entropy [kb/by]")
plt.xlabel("Radius [km]")

## Slice plot in 2D (cylindrical coordinates)

In [ ]:
slice2 = yt.SlicePlot(ds_2d,'theta',"entr",origin="native",center=[0,0,0],width=(400,'km'))
slice2.annotate_grids()
#slice2.hide_axes()
slice2.show()

## To fix it, one solution is using the fix-resolution-buffer method

In [ ]:
rmax = 2.e7
slice2d = yt.SlicePlot(ds_2d,"theta","entr",origin=('center','left','window'))
slc_frb = slice2d.data_source.to_frb((2*rmax,"cm"),1024,center=(0,0,0),height=(2*rmax,"cm"))
plt.figure(10,figsize=(6,9))
plt.imshow(slc_frb["entr"].d,extent=[-rmax,rmax,-rmax,rmax],
           interpolation='nearest',
           aspect=1.0,
           cmap='Spectral_r',
           origin='center')
plt.xlim([0,rmax])
plt.ylim([-rmax,rmax])
cbar = plt.colorbar(shrink=1.0)
cbar.ax.set_ylabel("Entropy [kb/by]",rotation=270,labelpad=25)
plt.xlabel("R [cm]")
plt.ylabel("Z [cm]")

## or use Kuo-Chuan's script 

In [ ]:
from scripts.slice2d import *

In [ ]:
fig = slice2d(ds_2d,"entr",rmax=2e7)

In [ ]:
fig = slice2d(ds_2d,"dens",rmax=2e7,clim=[1e9,1e15],take_log=True)

## ADding new derived fields

In [ ]:
slice = yt.SlicePlot(ds_3d,'z',"specific_angular_momentum_magnitude",width=(250,'km'))
slice.show()

In [ ]:
values = ds_3d.h.find_field_values_at_point(["dens","entr","ye  "],[2.7,0,0])
print("Entropy is ", values[1])

In [ ]:
def add_entropy(ds):
    from yt.units.dimensions import mass, energy, temperature
    from yt.units import cm
    ds.unit_registry.add('kB',1.3806488e-16,dimensions=energy/temperature,tex_repr=r'{\rm k_{B}}')
    ds.unit_registry.add('by',1.674e-24,dimensions=mass,tex_repr=r'{\rm baryon}')
    def _entr(field,data):
        entr = data["entr"]
        kb_by = yt.YTQuantity(1,'erg')/yt.YTQuantity(1,'K')/yt.YTQuantity(1,'g')*(1.3806488e-16/1.674e-24)
        return entr*kb_by
    ds.add_field("Entropy",function=_entr,units="kB/by",
                 display_name="Entropy",
                 dimensions=energy/temperature/mass)
    return ds

In [ ]:
ds_3d = add_entropy(ds_3d)

In [ ]:
values = ds_3d.h.find_field_values_at_point(["dens","Entropy","ye  "],[2.7,0,0])
print("Entropy is ", values[1])

### Adding gradient fields

In [ ]:
ds_3d.periodicity = (True, True, True)

In [ ]:
ds_3d.add_gradient_fields(("gas","Entropy"))

In [ ]:
slice = yt.SlicePlot(ds_3d,'z',"Entropy_gradient_magnitude",width=(250,'km'))

In [ ]:
slice.show()

## Working with Cylindrical data

In [ ]:
ds_2d.derived_field_list

In [ ]:
fig = slice2d(ds_2d,"velocity_magnitude",rmax=2e7,clim=[0,1e10],take_log=False)

In [ ]:
def _cyl_radial_velocity(field,data):
    r = data["r"]
    z = data["z"]
    velx = data["velx"]
    vely = data["vely"]
    phi = np.arctan(z/r)
    velr = np.cos(phi)*velx + np.sin(phi)*vely
    return velr

def _cyl_tangential_velocity(field,data):
    r = data["r"]
    z = data["z"]
    velx = data["velx"]
    vely = data["vely"]
    phi = np.arctan(z/r)
    tanr = -np.sin(phi)*velx + np.cos(phi)*vely
    return tanr

In [ ]:
ds_2d.add_field("radial_velocity",function=_cyl_radial_velocity,units='cm/s',sampling_type='cell')
ds_2d.add_field("tangential_velocity",function=_cyl_tangential_velocity,units='cm/s',sampling_type='cell')

In [ ]:
fig = slice2d(ds_2d,"radial_velocity",rmax=2e7,clim=[-1e9,1e9],take_log=False)

## Filtering Data

In [ ]:
sp = ds_2d.sphere('max',(1000,'km'))
pns = sp.cut_region(['obj["density"] > 1e11'])

In [ ]:
cm = pns["cell_mass"]
print(cm.sum().in_units("msun"))

In [ ]:
ad = ds_3d.all_data()
shocked_region =  ad.cut_region(['(obj["dens"] < 1e11) & (obj["entr"] > 8)'])

In [ ]:
shocked_region["dens"]

In [ ]:
shocked_region["radius"]

In [ ]:
slice = yt.SlicePlot(ds_3d,'z',"entr",width=(250,'km'),data_source=shocked_region)

In [ ]:
slice.set_log('entr',False)
slice.show()

## Saving reloadable data

Method (1)

Data from geometric data containers can be saved with the save_as_dataset`() function.

In [ ]:
fn = shocked_region.save_as_dataset(fields=["entr","dens"])

In [ ]:
sh = yt.load("ccsn3d_hdf5_plt_cnt_0656_cut_region.h5")

In [ ]:
ad = sh.all_data()

In [ ]:
print(ad["grid", "x"])
print(ad["entr"])

Note that because field data queried from geometric containers is returned as unordered 1D arrays, data container datasets are treated, effectively, as particle data. Thus, 3D indexing of grid data from these datasets is not possible.

Method (2)

In [ ]:
slc = ds_3d.slice('z', 0, data_source=shocked_region)

In [ ]:
slc_frb = slc.to_frb((800.0,"km"),800)

In [ ]:
fn = slc_frb.save_as_dataset(fields=["entr","dens","ye  "])
print("DATASET: {} is saved.".format(fn))

In [ ]:
frb = yt.load(fn)

In [ ]:
dens = frb.data["dens"]
entr = frb.data["entr"]

In [ ]:
print(entr)

In [ ]:
dx = 1e5 # [cm]
xmin = -4.e7
xmax = 4.e7
xx = np.linspace(xmin + 0.5*dx,xmax - 0.5*dx,800)
yy = np.linspace(xmin + 0.5*dx,xmax - 0.5*dx,800)
plt.figure(1,figsize=(12,12))
plt.imshow(entr, extent=[xmin,xmax,xmin,xmax],interpolation='bicubic',aspect=1.0,origin='center', cmap="Spectral_r")
plt.xlabel("X [cm]")
plt.ylabel("Y [cm]")
plt.xlim([-2e7,2e7])
plt.ylim([-2e7,2e7])
cbar =plt.colorbar(shrink=0.8)
plt.clim([0,30])
plt.contour(dens,[1e11],colors=('y'),extent=[xmin,xmax,xmin,xmax])
plt.axis('off')

# Volume rendering
also see: https://yt-project.org/docs/dev/visualizing/volume_rendering.html

In [ ]:
sc = yt.create_scene(ds_3d, field="entr")
sc.camera.set_width((300,'km'))

source = sc.get_source(0)
source.set_field('entr')
source.set_log(False)
bounds = (10, 30)

# Since this rendering is done in log space, the transfer function needs
# to be specified in log space.
tf = yt.ColorTransferFunction(bounds)
tf.sample_colormap(25, w=.5, colormap='autumn')

source.tfh.tf = tf
source.tfh.bounds = bounds
source.tfh.plot('transfer_function.png', profile_field='entr')

sc.save('rendering.png', sigma_clip=6)